In [1]:
from bancor_research import DEFAULT
from bancor_research.bancor_simulator.v3.spec.network import BancorDapp as sBancorDapp
from bancor_research.bancor_emulator.v3.spec.network import BancorDapp as eBancorDapp

# Recall from an earlier chapter that we already defined the whitelisted_tokens as follows.
whitelisted_tokens = {tkn_name : DEFAULT.WHITELIST[tkn_name] for tkn_name in DEFAULT.WHITELIST if tkn_name in ['eth', 'link', 'wbtc']}

# There are other possible configuration settings available, however for the present purpose we will use the defaults.
dapps = [BancorDapp(whitelisted_tokens=whitelisted_tokens) for BancorDapp in [sBancorDapp, eBancorDapp]]

def assertAndDisplay(decimals = -1):
    frames = [v3.describe(decimals) for v3 in dapps]
    diff = frames[0].compare(frames[1])
    assert diff.empty, diff
    return frames[0]

for v3 in dapps:
    v3.create_user('Trader')
    v3.create_user('David')

    v3.set_user_balance(tkn_amt='101', tkn_name='eth', user_name='Alice')
    v3.set_user_balance(tkn_amt='101', tkn_name='wbtc', user_name='Bob')
    v3.set_user_balance(tkn_amt='10001', tkn_name='link', user_name='Charlie')
    v3.set_user_balance(tkn_amt='2000', tkn_name='bnt', user_name='Trader')
    v3.set_user_balance(tkn_amt='1', tkn_name='eth', user_name='Trader')
    v3.set_user_balance(tkn_amt='3', tkn_name='link', user_name='Trader')
    v3.set_user_balance(tkn_amt='1000', tkn_name='bnt', user_name='David')

    v3.deposit(tkn_amt='100', tkn_name='eth', user_name='Alice')
    v3.deposit(tkn_amt='10000', tkn_name='link', user_name='Charlie')
    v3.deposit(tkn_amt='100', tkn_name='wbtc', user_name='Bob')
    v3.deposit(tkn_amt='1', tkn_name='eth', user_name='Alice')
    v3.deposit(tkn_amt='1', tkn_name='link', user_name='Charlie')
    v3.deposit(tkn_amt='1', tkn_name='wbtc', user_name='Bob')

    for tkn_name in whitelisted_tokens:
        v3.enable_trading(tkn_name)

    v3.trade(tkn_amt='2000', source_token='bnt',  target_token='link', user_name='Trader', timestamp=0)
    v3.trade(tkn_amt='302.9981', source_token='link',  target_token='bnt', user_name='Trader', timestamp=0)
    v3.trade(tkn_amt='1', source_token='eth',  target_token='wbtc', user_name='Trader', timestamp=0)

# Staking BNT

In Bancor 3, assume that BNT liquidity providers are receiving protocol bnBNT tokens in return for destroying BNT. The calculation is essentially identical to the standard pool token method, and its outcome is relatively easy to understand. However, there are some nuances to the process that should be highlighted.

For this section, we can consider the state of the system after the initial bootstrapping and first trading activity described earlier. The system snapshot is as follows:

In [2]:
assertAndDisplay(decimals=4)

bnt         eth        link  \
1 Account Alice                         0.0000      0.0000      0.0000   
          Bob                           0.0000      0.0000      0.0000   
          Charlie                       0.0000      0.0000      0.0000   
          David                      1000.0000      0.0000      0.0000   
          Trader                     1977.6153      0.0000      0.0019   
2 Pool    a: TKN Staked Balance              0    101.0000  10003.4242   
          b: TKN Trading Liquidity           0     21.0000   3336.3314   
          c: BNT Trading Liquidity           0  19055.2381  20013.9939   
          d: BNT Current Funding             0  20007.6190  20015.9807   
          e: Spot Rate                       0    907.3923      5.9988   
          f: Average Rate                    0   1000.0000      6.0000   
          g: Average Inverse Rate            0      0.0010      0.1667   
3 Network Master Vault              60022.3847    102.0000  10003.9981   
          Protection Vault              0.0000      0.0000      0.0000   
          Protocol Equity                    0           0           0   
          Rewards Vault                      0           0           0   

                                          wbtc       bnbnt     bneth  \
1 Account Alice                         0.0000      0.0000  101.0000   
          Bob                           0.0000      0.0000    0.0000   
          Charlie                       0.0000      0.0000    0.0000   
          David                         0.0000      0.0000    0.0000   
          Trader                        0.0557      0.0000    0.0000   
2 Pool    a: TKN Staked Balance       101.0005           0         0   
          b: TKN Trading Liquidity      1.1943           0         0   
          c: BNT Trading Liquidity  20940.8835           0         0   
          d: BNT Current Funding    20000.0000           0         0   
          e: Spot Rate              17534.2111           0         0   
          f: Average Rate           16000.0000           0         0   
          g: Average Inverse Rate       0.0001           0         0   
3 Network Master Vault                100.9443           0         0   
          Protection Vault              0.0000           0         0   
          Protocol Equity                    0  60000.0000    0.0000   
          Rewards Vault                      0      0.0000    0.0000   

                                        bnlink    bnwbtc    vbnt  
1 Account Alice                         0.0000    0.0000  0.0000  
          Bob                           0.0000  101.0000  0.0000  
          Charlie                   10001.0000    0.0000  0.0000  
          David                         0.0000    0.0000  0.0000  
          Trader                        0.0000    0.0000  0.0000  
2 Pool    a: TKN Staked Balance              0         0       0  
          b: TKN Trading Liquidity           0         0       0  
          c: BNT Trading Liquidity           0         0       0  
          d: BNT Current Funding             0         0       0  
          e: Spot Rate                       0         0       0  
          f: Average Rate                    0         0       0  
          g: Average Inverse Rate            0         0       0  
3 Network Master Vault                       0         0       0  
          Protection Vault                   0         0       0  
          Protocol Equity               0.0000    0.0000       0  
          Rewards Vault                 0.0000    0.0000       0

In [3]:
timestamp = 0
for v3 in dapps:
    v3.deposit(tkn_amt='1000', tkn_name='bnt',  user_name='David', timestamp=timestamp)
assertAndDisplay(decimals=4)

bnt         eth        link  \
1 Account Alice                         0.0000      0.0000      0.0000   
          Bob                           0.0000      0.0000      0.0000   
          Charlie                       0.0000      0.0000      0.0000   
          David                         0.0000      0.0000      0.0000   
          Trader                     1977.6153      0.0000      0.0019   
2 Pool    a: TKN Staked Balance              0    101.0000  10003.4242   
          b: TKN Trading Liquidity           0     21.0000   3336.3314   
          c: BNT Trading Liquidity           0  19055.2381  20013.9939   
          d: BNT Current Funding             0  20007.6190  20015.9807   
          e: Spot Rate                       0    907.3923      5.9988   
          f: Average Rate                    0   1000.0000      6.0000   
          g: Average Inverse Rate            0      0.0010      0.1667   
3 Network Master Vault              60022.3847    102.0000  10003.9981   
          Protection Vault              0.0000      0.0000      0.0000   
          Protocol Equity                    0           0           0   
          Rewards Vault                      0           0           0   

                                          wbtc       bnbnt     bneth  \
1 Account Alice                         0.0000      0.0000  101.0000   
          Bob                           0.0000      0.0000    0.0000   
          Charlie                       0.0000      0.0000    0.0000   
          David                         0.0000    999.6068    0.0000   
          Trader                        0.0557      0.0000    0.0000   
2 Pool    a: TKN Staked Balance       101.0005           0         0   
          b: TKN Trading Liquidity      1.1943           0         0   
          c: BNT Trading Liquidity  20940.8835           0         0   
          d: BNT Current Funding    20000.0000           0         0   
          e: Spot Rate              17534.2111           0         0   
          f: Average Rate           16000.0000           0         0   
          g: Average Inverse Rate       0.0001           0         0   
3 Network Master Vault                100.9443           0         0   
          Protection Vault              0.0000           0         0   
          Protocol Equity                    0  59000.3932    0.0000   
          Rewards Vault                      0      0.0000    0.0000   

                                        bnlink    bnwbtc      vbnt  
1 Account Alice                         0.0000    0.0000    0.0000  
          Bob                           0.0000  101.0000    0.0000  
          Charlie                   10001.0000    0.0000    0.0000  
          David                         0.0000    0.0000  999.6068  
          Trader                        0.0000    0.0000    0.0000  
2 Pool    a: TKN Staked Balance              0         0         0  
          b: TKN Trading Liquidity           0         0         0  
          c: BNT Trading Liquidity           0         0         0  
          d: BNT Current Funding             0         0         0  
          e: Spot Rate                       0         0         0  
          f: Average Rate                    0         0         0  
          g: Average Inverse Rate            0         0         0  
3 Network Master Vault                       0         0         0  
          Protection Vault                   0         0         0  
          Protocol Equity               0.0000    0.0000         0  
          Rewards Vault                 0.0000    0.0000         0

The most important nuance to be aware of is that BNT liquidity providers are supporting the BNT liquidity of the whole protocol, rather than any specific pool. As a result, there is no spot price or moving average. Further, bnBNT pool tokens are not created when a BNT liquidity provider adds their tokens to the protocol; there is no change in the vault balance of BNT, or its staked balance (save for one extreme edge case). The provision of BNT by users is best thought of as a private exchange of BNT directly for protocol-owned bnBNT.

To demonstrate, assume a fourth participant, David, wishes to provide 1,000 BNT liquidity to the protocol. The only calculations the protocol must perform are to value the BNT David is providing. The staking ledger is reporting a total of 6,002.3599 BNT, and the bnBNT pool token supply is 6,000 bnBNT. Therefore, the bnBNT/BNT exchange rate is 0.9996068, and David’s 1,000 BNT is worth 999.60683797 bnBNT. When David confirms this transaction, the protocol simply transfers this amount of bnBNT to him, from its own balance; the BNT that David provided is burned immediately.

Note that the only observable change in the system is that the protocol owns less of the total BNT; outside of the system, the ERC20 contract supply of BNT will have diminished slightly. David also receives vBNT, the Bancor governance token, at a 1:1 rate with respect to the bnBNT pool tokens (see the following section). vBNT is staked in the governance contract to take part in Bancor’s decision making process; however, it must also be returned when a user exits from the protocol.